<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/veronica-thesis/blob/master/malware.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Packages

In [ ]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt
from tensorflow.keras import models,layers
from tensorflow.keras.utils import Sequence
#from tensorflow.python.keras.utils import data_utils
import math
import os
import pandas
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv3D,Conv2D,InputLayer,MaxPooling3D,Dropout,Flatten,MaxPooling2D,BatchNormalization,LayerNormalization


In [ ]:

def conv(x):
    if x=='??':
        return -1
    else:
        return int(x,16)


def getImage(dir,resized_dir,filename):
    f=open(filename)
    getImage.count+=1
    if getImage.count%100 == 0:
       print("processed {} images".format(getImage.count))
    rest=f.read()
    lines=rest.splitlines()
    img=[]
    for line in lines:
        el=line.split()
        el=list(map(conv,el))
        if(len(el)==17):
            img.append(el[1:])
    img=np.asarray(img,dtype=np.uint8)
    img=Image.fromarray(img,'L')
    img=img.resize((256,256),resample=Image.LANCZOS)
    img=np.array(img)
    img=img.reshape(img.shape[0],img.shape[1],1)
    f.close()
    return img

class ImageSequence(Sequence):
    def __init__(self, x_set, y_set,dir, resized_dir,batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.dir=dir
        self.resized_dir=resized_dir

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

        return np.array([getImage(dir,resized_dir,filename) for filename in batch_x]),batch_y


In [ ]:
getImage.count=0
batch_size=4000
data=pandas.read_csv("data/data-4000.csv",header=None)
names=data.iloc[:,0].to_numpy()
classes=data.iloc[:,1].to_numpy()-1
##data=pandas.read_csv("data/data.2.csv",header=None)
##names=np.append(names,data.iloc[:,0].to_numpy())
##classes=np.append(classes,data.iloc[:,1].to_numpy)
#dir="c:\\Users\\hikma\\Downloads\\"
dir="data\\train\\"

train_dataset=ImageSequence(names,classes,dir,batch_size)

print("getting first batch")
(f,l)=train_dataset.__getitem__(0)
f.shape
l

In [ ]:
f.shape

In [ ]:
def createModelYuan():
    model = Sequential()
    model.add(InputLayer(input_shape=(256,256,1)))
    #model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255))
   
    model.add(Conv2D(64, kernel_size = (3,3),strides=(1,1), padding='same',activation = 'relu'))
    model.add(Conv2D(64, kernel_size = (3,3),strides=(1,1), padding='same',activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(128, kernel_size = (3,3),strides=(1,1), padding='same',activation = 'relu'))
    model.add(Conv2D(128, kernel_size = (3,3),strides=(1,1), padding='same',activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    

    model.add(Conv2D(256, kernel_size = (3,3),strides=(1,1), padding='same',activation = 'relu'))
    model.add(Conv2D(256, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(Conv2D(256, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))


    model.add(Conv2D(512, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(Conv2D(512, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(Conv2D(512, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(512, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(Conv2D(512, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(Conv2D(512, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))


    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(9, activation = 'softmax', name = 'Output'))
    return model


In [ ]:

model=createModelYuan()
model.summary()

## Optimization

Keras can use many optimization method. In this notebook we use the __Adam__ method which can be described loosely as __adaptive__ gradient descent.

Also since the labels are __NOT__ in one_hot_encoding we use the "Sparse" version of the crossentropy loss: __SparseCategoricalCrossentropy__. Finally, if we don't specify from_logits=False then the loss function would compute softwmax before computing the loss. Since we are computing softwmax in our model already we turn this step off by specifying from_logits=False

In [ ]:
# if we don't use softmax in the last layer, i.e. if the output of the
# model is NOT probabilities then use from_logits=True
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

#history = model.fit(img_train,label_train, batch_size=128,epochs=10, 
#                    validation_data=(img_test, label_test))
#history=model.fit(train_dataset,batch_size=batch_size,epochs=3)
model.load_weights("weights-2000.h5")
history=model.fit(f,l,batch_size=32,epochs=5)


In [ ]:
model.save_weights("weights-4000.h5")

### Testing the Accuracy

In [ ]:
data=pandas.read_csv("data/data.6.csv",header=None)
# test dataset
n=800
names=data.iloc[0:n,0].to_numpy()
classes=data.iloc[0:n,1].to_numpy()-1
test_dataset=ImageSequence(names,classes,dir,32)
_,test_accuracy=model.evaluate(test_dataset)